In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler, KMeansSMOTE, SVMSMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# kagglehub.dataset_download("gabrielsantello/cars-purchase-decision-dataset")

car = pd.read_csv("E:\Workings\Thesis\Percentile v SMOTE\Datasets smote\car_data.csv")

<>:3: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
C:\Users\danis\AppData\Local\Temp\ipykernel_17200\4286661620.py:3: SyntaxWarning: invalid escape sequence '\W'
  car = pd.read_csv("E:\Workings\Thesis\Percentile v SMOTE\Datasets smote\car_data.csv")


In [3]:
del car['User ID']

In [4]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Gender        1000 non-null   object
 1   Age           1000 non-null   int64 
 2   AnnualSalary  1000 non-null   int64 
 3   Purchased     1000 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 31.4+ KB


In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
label_encoder = LabelEncoder()
car['Gender'] = label_encoder.fit_transform(car['Gender'])

In [7]:
car

,Gender,Age,AnnualSalary,Purchased
0,1,35,20000,0
1,1,40,43500,0
2,1,49,74000,0
3,1,40,107500,1
4,1,25,79000,0
...,...,...,...,...
995,1,38,59000,0
996,0,47,23500,0
997,0,28,138500,1
998,0,48,134000,1


In [8]:
from imblearn.datasets import make_imbalance
from sklearn.model_selection import train_test_split
from collections import Counter

In [9]:
car['Purchased'].value_counts()

Purchased
0    598
1    402
Name: count, dtype: int64

In [10]:
df_resampled, y_resampled = make_imbalance(car, car['Purchased'], sampling_strategy={ 0 : 598, 1: 200},random_state=42)
df_resampled2 = df_resampled.copy()     # saving same data to be used for our proposed method 
print("Original class distribution:", Counter(y_resampled))
del df_resampled['Purchased']

Original class distribution: Counter({0: 598, 1: 200})


In [11]:
# Function to calculate the required metrics 
def get_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
    auc = roc_auc_score(y_true, y_prob[:, 1])
    return accuracy, precision, recall, f1, auc

# Define classifiers
classifiers = {
    'GaussianNB': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'RandomForest': RandomForestClassifier(random_state=42)
}

# Function to classify and store metrics
def classify_and_store(classifiers, X_train, y_train, X_test, y_test, oversampler=None):
    if oversampler:
        # Apply oversampling
        X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
    else:
        X_train_resampled, y_train_resampled = X_train, y_train
    
    # Store the metrics for each classifier
    classifier_metrics = {}
    
    for name, clf in classifiers.items():
        # Train the classifier
        clf.fit(X_train_resampled, y_train_resampled)
        
        # Predict on the test set
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)
        
        # Get metrics
        accuracy, precision, recall, f1, auc = get_metrics(y_test, y_pred, y_prob)
        classifier_metrics[name] = {
            'accuracy': accuracy,
            'precision_0': precision[0],
            'precision_1': precision[1],
            'recall_0': recall[0],
            'recall_1': recall[1],
            'f1_0': f1[0],
            'f1_1': f1[1],
            'auc_0': auc if len(np.unique(y_test)) == 2 else None,
            'auc_1': auc if len(np.unique(y_test)) == 2 else None
        }
    
    return classifier_metrics


# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(df_resampled, y_resampled, test_size=0.2, random_state=42)

# Standardizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 1: Perform baseline evaluation (without oversampling) and store results
oversamplers = {
    'RandomOverSampler': RandomOverSampler(random_state=42, sampling_strategy='minority'),
    'SMOTE': SMOTE(random_state=42, sampling_strategy='minority'),
    'ADASYN': ADASYN(random_state=42, sampling_strategy='minority'),
    'KMeansSMOTE': KMeansSMOTE(random_state=42, sampling_strategy='minority'),
    'SVMSMOTE': SVMSMOTE(random_state=42, sampling_strategy='minority')
}

# Store all results in a single DataFrame
all_results = []

# Step 2: Baseline (no oversampling)
baseline_metrics = classify_and_store(classifiers, X_train, y_train, X_test, y_test)
for clf_name, m in baseline_metrics.items():
    all_results.append({
        'Classifier': clf_name,
        'Oversampler': 'None',
        'Accuracy': m['accuracy'],
        'Precision(0)': m['precision_0'],
        'Precision(1)': m['precision_1'],
        'Recall (0)': m['recall_0'],
        'Recall (1)': m['recall_1'],
        'F1(0)': m['f1_0'],
        'F1(1)': m['f1_1'],
        'AUC(0)': m['auc_0'],
        'AUC(1)': m['auc_1']
    })

# Step 3: Loop over classifiers and then oversamplers
for clf_name, clf in classifiers.items():
    for sampler_name, oversampler in oversamplers.items():
        oversampled_metrics = classify_and_store({clf_name: clf}, X_train, y_train, X_test, y_test, oversampler)
        
        # Now m will be the dictionary with metrics for the classifier
        m = oversampled_metrics[clf_name]
        
        all_results.append({
            'Classifier': clf_name,
            'Oversampler': sampler_name,
            'Accuracy': m['accuracy'],
            'Precision(0)': m['precision_0'],
            'Precision(1)': m['precision_1'],
            'Recall (0)': m['recall_0'],
            'Recall (1)': m['recall_1'],
            'F1(0)': m['f1_0'],
            'F1(1)': m['f1_1'],
            'AUC(0)': m['auc_0'],
            'AUC(1)': m['auc_1']
        })

# Convert to DataFrame
results_df = pd.DataFrame(all_results)


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1955: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=3
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1955: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=3
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1955: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=3
  warnings.warn(


_______________________________________________________________________________________________________________________________________________________________________

In [12]:
car[car['Purchased']==1].corr().abs().sum().sort_values()

Purchased       0.000000
Gender          1.151233
AnnualSalary    1.384483
Age             1.433290
dtype: float64

In [13]:
df_resampled2[df_resampled2['Purchased']==1].count()

Gender          200
Age             200
AnnualSalary    200
Purchased       200
dtype: int64

In [14]:
483-200

283

In [15]:
minority_samples = df_resampled2[df_resampled2['Purchased']==1]
random_values = minority_samples['Age'].sample(n=283, replace=True)
random_values = random_values.reset_index()
random_values = pd.DataFrame(random_values)
del random_values['index']
random_values

,Age
0,53
1,52
2,53
3,27
4,35
...,...
278,59
279,46
280,38
281,48


In [16]:
car.columns

Index(['Gender', 'Age', 'AnnualSalary', 'Purchased'], dtype='object')

In [17]:
random_values['Gender'] = np.nan
random_values['AnnualSalary'] = np.nan
random_values['Purchased'] = 1

In [18]:
random_values

,Age,Gender,AnnualSalary,Purchased
0,53,NaN,NaN,1
1,52,NaN,NaN,1
2,53,NaN,NaN,1
3,27,NaN,NaN,1
4,35,NaN,NaN,1
...,...,...,...,...
278,59,NaN,NaN,1
279,46,NaN,NaN,1
280,38,NaN,NaN,1
281,48,NaN,NaN,1


In [19]:
def calculate_percentiles(nums):
    indexed_nums = [(num, i) for i, num in enumerate(nums)]
    sorted_nums = []
    for num_index in indexed_nums:
        inserted = False
        for i, sorted_num_index in enumerate(sorted_nums):
            if num_index[0] < sorted_num_index[0]:
                sorted_nums.insert(i, num_index)
                inserted = True
                break
        if not inserted:
            sorted_nums.append(num_index)
    length = len(sorted_nums)
    percentiles = [0] * length
    for i, num_index in enumerate(sorted_nums):
        original_index = num_index[1]
        percentile = ((i + 1) / length) * 100
        percentiles[original_index] = percentile
    return percentiles

def dataframe_to_percentiles(df):
    df_percentiles = df.copy()
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            df_percentiles[column] = calculate_percentiles(df[column].tolist())
    return df_percentiles


In [20]:
random_values['Age'] = calculate_percentiles(random_values['Age'])
random_values['Age'] = 'P' + random_values['Age'].round().astype(str,errors='ignore')

In [21]:
random_values['Age'].unique()

array(['P69.0', 'P64.0', 'P0.0', 'P8.0', 'P4.0', 'P94.0', 'P90.0',
       'P60.0', 'P91.0', 'P70.0', 'P84.0', 'P14.0', 'P80.0', 'P30.0',
       'P29.0', 'P55.0', 'P25.0', 'P71.0', 'P99.0', 'P17.0', 'P34.0',
       'P77.0', 'P35.0', 'P78.0', 'P22.0', 'P65.0', 'P49.0', 'P26.0',
       'P81.0', 'P7.0', 'P72.0', 'P31.0', 'P85.0', 'P42.0', 'P3.0',
       'P50.0', 'P92.0', 'P19.0', 'P74.0', 'P27.0', 'P2.0', 'P61.0',
       'P36.0', 'P86.0', 'P51.0', 'P98.0', 'P66.0', 'P75.0', 'P87.0',
       'P43.0', 'P95.0', 'P23.0', 'P67.0', 'P79.0', 'P56.0', 'P15.0',
       'P44.0', 'P82.0', 'P9.0', 'P21.0', 'P18.0', 'P10.0', 'P37.0',
       'P62.0', 'P45.0', 'P38.0', 'P100.0', 'P68.0', 'P63.0', 'P52.0',
       'P46.0', 'P57.0', 'P96.0', 'P32.0', 'P1.0', 'P47.0', 'P5.0',
       'P39.0', 'P53.0', 'P40.0', 'P12.0', 'P11.0', 'P16.0', 'P24.0',
       'P20.0', 'P73.0', 'P83.0', 'P88.0', 'P58.0', 'P76.0', 'P28.0',
       'P97.0', 'P41.0', 'P6.0', 'P93.0', 'P13.0', 'P48.0', 'P89.0',
       'P59.0', 'P33.0', 'P54

In [22]:
carP = dataframe_to_percentiles(minority_samples)
carP = 'P' + carP.round().astype(str,errors='ignore')
carP['Purchased'] = 1

In [23]:
minority_samples

,Gender,Age,AnnualSalary,Purchased
34,0,44,113500,1
884,0,52,41500,1
232,1,48,33000,1
62,1,49,88000,1
298,1,50,29500,1
...,...,...,...,...
960,1,32,100000,1
661,1,34,112000,1
919,0,52,21000,1
794,1,50,87500,1


In [24]:
ZERO = pd.concat([carP,random_values], ignore_index=True)
ZERO

,Gender,Age,AnnualSalary,Purchased
0,P0.0,P30.0,P65.0,1
1,P1.0,P62.0,P25.0,1
2,P52.0,P44.0,P14.0,1
3,P52.0,P52.0,P44.0,1
4,P53.0,P56.0,P12.0,1
...,...,...,...,...
478,NaN,P90.0,NaN,1
479,NaN,P34.0,NaN,1
480,NaN,P16.0,NaN,1
481,NaN,P49.0,NaN,1


In [25]:


# Step 1: Identify unique values of 'pH'
unique_size = carP['Age'].unique()

# Step 2: Create a dictionary to store non-missing values for each variable
# Initialize the dictionary
imputation_info = {}

# Iterate over each variable (excluding 'pH') that has missing values
for column in carP.columns:
    if column != 'Age' and ZERO[column].isna().sum() > 0:
        imputation_info[column] = {}
        
        # Iterate over each unique value of 'pH'
        for size_value in unique_size:
            # Get the non-missing values of the variable where 'pH' is equal to the current pH_value
            non_missing_values = carP.loc[carP['Age'] == size_value, column].dropna().values
            imputation_info[column][size_value] = non_missing_values

In [26]:
# import random
import statistics
def fill_missing_values2(row, imputation_info):
    # For each column, check if it has a missing value
    for column in imputation_info:
        if pd.isna(row[column]):
            size_value = row['Age']  # Get the corresponding pH value for the row
            if size_value in imputation_info[column]:
                possible_values = imputation_info[column][size_value]
                if len(possible_values) > 0:
                    row[column ] = statistics.mode(possible_values)
                    # row[column] = random.choice(possible_values)

    return row

myMode = ZERO.apply(lambda row: fill_missing_values2(row, imputation_info), axis=1)

In [27]:
myMode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483 entries, 0 to 482
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Gender        483 non-null    object
 1   Age           483 non-null    object
 2   AnnualSalary  483 non-null    object
 3   Purchased     483 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 15.2+ KB


In [28]:
print(myMode.shape)
print(ZERO.shape)

(483, 4)
(483, 4)


In [29]:
percentiles = [*range(0,101, 1)]
mapping_data = {}

columns_to_impute = car.columns.difference(['Purchased'])
for variable in columns_to_impute:
    mapping_data[variable] = [car[car['Purchased']==1][variable].quantile(p / 100) for p in percentiles]

# Create the mapping DataFrame
mapping_df = pd.DataFrame(mapping_data, index=[f'P{p}' for p in percentiles])
print("Mapping DataFrame:")
mapping_df

Mapping DataFrame:


,Age,AnnualSalary,Gender
P0,27.00,20000.0,0.0
P1,28.01,21505.0,0.0
P2,30.00,22510.0,0.0
P3,31.03,23015.0,0.0
P4,32.00,24500.0,0.0
...,...,...,...
P96,61.00,147500.0,1.0
P97,61.00,148500.0,1.0
P98,62.00,149500.0,1.0
P99,63.00,150500.0,1.0


In [30]:
def convert_percentiles_to_values(mdf, mapping_df):
    columns_to_impute = mdf.columns.difference(['Purchased'])

    for column in columns_to_impute:
        mdf[column] = mdf[column].apply(lambda x: mapping_df.loc[x, column] if isinstance(x, str) and x.startswith('P') else x)
    return mdf

In [31]:
def path_to_revert(now, then):
    now = now.where(then.isna(), then)
    now = now.replace(r'^(P\d+)\.0$', r'\1', regex=True)
    return now

In [32]:
a = path_to_revert(myMode, ZERO)
a = convert_percentiles_to_values(a, mapping_df)

In [33]:
a

,Gender,Age,AnnualSalary,Purchased
0,0.0,44.00,109150.0,1
1,0.0,51.00,43500.0,1
2,0.0,48.00,33570.0,1
3,0.0,49.00,88000.0,1
4,0.0,49.56,31560.0,1
...,...,...,...,...
478,0.0,60.00,30500.0,1
479,0.0,46.00,22510.0,1
480,0.0,39.00,80500.0,1
481,1.0,48.00,26000.0,1


In [34]:
percent = pd.concat([a, df_resampled2[df_resampled2['Purchased']==0][0:483]], ignore_index=True)

In [35]:
percent['Purchased'].value_counts()

Purchased
1    483
0    483
Name: count, dtype: int64

In [36]:
X_percent = percent.drop(columns=['Purchased'])  # Replace with actual target column name
y_percent = percent['Purchased']  # Replace with actual target column name

# Standardize the 'percent' data as well
X_percent = scaler.fit_transform(X_percent)

def evaluate_custom_oversampling(X_train, y_train, X_test, y_test, classifiers):
    custom_results = []
    
    # Iterate over classifiers first
    for clf_name, clf in classifiers.items():
        # Train the classifier
        clf.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)
        
        # Get metrics
        accuracy, precision, recall, f1, auc = get_metrics(y_test, y_pred, y_prob)
        
        # Append the result
        custom_results.append({
            'Classifier': clf_name,               # Classifier first
            'Oversampler': 'PERCENTILES',              # Oversampler (Custom in this case)
            'Accuracy': accuracy,
            'Precision(0)': precision[0],
            'Precision(1)': precision[1],
            'Recall (0)': recall[0],
            'Recall (1)': recall[1],
            'F1(0)': f1[0],
            'F1(1)': f1[1],
            'AUC(0)': auc,
            'AUC(1)': auc
        })
    
    # Convert to DataFrame and return
    return pd.DataFrame(custom_results)



# Evaluate on custom oversampled data (the 'percent' dataset)
custom_results_df = evaluate_custom_oversampling(X_percent, y_percent, X_test, y_test, classifiers)


# Output: Print or review the custom oversampling results
custom_results_df

,Classifier,Oversampler,Accuracy,Precision(0),Precision(1),Recall (0),Recall (1),F1(0),F1(1),AUC(0),AUC(1)
0,GaussianNB,PERCENTILES,0.80625,0.988372,0.594595,0.739130,0.977778,0.845771,0.739496,0.951304,0.951304
1,KNN,PERCENTILES,0.86875,0.989583,0.687500,0.826087,0.977778,0.900474,0.807339,0.946570,0.946570
2,RandomForest,PERCENTILES,0.85000,0.959596,0.672131,0.826087,0.911111,0.887850,0.773585,0.941256,0.941256


In [ ]:

combined_df = pd.concat([results_df, custom_results_df], ignore_index=True)

# Save to Excel
combined_df.to_excel("car3.xlsx", index=False)
